### Retrieval Augmented Generation Negative Scenarios



### Notebook Details
The following Python Notebook will guide you through using the RAG APIs.

#### Assumptions/Prerequisites
- You have your own Azure storage account with the data you want to process uploaded and ready.
- The files are uploaded to a directory (not the container root).
- You have the connection string for the storage account.
- Your storage is accessible from both your local KPMG Laptop and the API compute in the Core VNET.
- You have an API key with access to the RAG product in APIM.
- You have an API key with access to the Azure OpenAI Completions/Embeddings API in APIM.
- You have a user configuration file that specifies that you have access to the "text-embedding-3-large" with an adequate quota to where you will not be throttled by the embeddings API


#### Steps
- Create a Catalog (a logical container for the data you want to index).
- Create a SAS URI that points to the data source(s) you want to process.
- Associate the SAS URI with the Catalog by creating a Catalog Source.
- Create an Ingestion Job:
  - Copy the files to a working directory inside the API private storage.
  - Extract the markdown content from all the copied files.
  - Chunk the markdown into smaller pieces.
  - Generate vectors or embeddings of the chunked contents.
  - Index a JSON representation of the chunks and vectors.
- Query the index to retrieve relevant documents.

After completing these steps, the relevant document chunks' contents can be passed to an LLM (Language Model) of your choice for summarization and analysis.

### Install the required packages

In [1]:
%pip install python-dotenv
%pip install requests
%pip install azure-storage-blob
%pip install tabulate

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### Load the configuration settings

In [2]:
from dotenv import dotenv_values, load_dotenv
import sys
sys.path.append('../common')
from helper import Helper

load_dotenv()

secrets = dotenv_values("secrets.env")
api_key = secrets.get("API_KEY")
local_env = secrets.get('LOCAL_URL') or secrets.get("APIM_NAME")
environment = secrets.get("APIM_NAME")
account_connection_string = secrets.get("AZURE_STORAGE_CONNECTION_STRING")
region_override = secrets.get("REGION_OVERRIDE")

ragHelper = Helper(local_env, "rag/kpmg", api_key, region_override)
deploymentsHelper = Helper(environment, "genai/azure/deployments", api_key, region_override)

In [3]:
# clean up any previous runs
ragHelper.log('Clean up from previous tests')
response = ragHelper.get("/catalog")

assert response.status_code == 200

if (len(response.json["value"])):
    for catalog in response.json["value"]:
        response = ragHelper.delete(f"/catalog/{catalog['catalogId']}")
        assert response.status_code == 200

Clean up from previous tests
Request: GET https://api.qa.workbench.kpmg/rag/kpmg/catalog
Response: 200
{'value': []}



### Catalog Tests

##### Create a new catalog

In [ ]:
ragHelper.log('Create a catalog for QA-Regression scenarios')

import datetime
 
catalogName = f'Regression Catalog: {datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%S")}'

ragHelper.log("Catalog Name: " + catalogName + '\n')

response = ragHelper.post("/catalog", {
    "displayName": catalogName
})

assert response.status_code == 200

catalogId = response.json["catalogId"]

ragHelper.log("Created Catalog Id: " + catalogId)

ragHelper.log('----------------------------------------------------------End of test----------------------------------------------------------')

Create a catalog for QA-Regression scenarios
Catalog Name: Regression Catalog: 2025-04-11T14:57:05

Request: POST https://api.qa.workbench.kpmg/rag/kpmg/catalog
Response: 200
{'catalogId': 'd955e43b-920d-4d74-9c7b-13dc6b0c3498'}

Created Catalog Id: d955e43b-920d-4d74-9c7b-13dc6b0c3498


##### Create a new catalog with existing display name

In [ ]:
#Using  catalog created in above step
ragHelper.log('Test - Create a new catalog with existing display name')

ragHelper.log("Catalog Name: " + catalogName + '\n')

response = ragHelper.post("/catalog", {
    "displayName": catalogName
})

assert response.status_code == 409
assert (f"A catalog with name {catalogName} already exists.") in response.json['message'].replace("'",'')

ragHelper.log('----------------------------------------------------------End of test----------------------------------------------------------')

Test - Create a new catalog with existing display name
Catalog Name: Regression Catalog: 2025-04-11T14:57:05

Request: POST https://api.qa.workbench.kpmg/rag/kpmg/catalog
Response: 409
{'message': 'A catalog with name Regression Catalog: 2025-04-11T14:57:05 already exists.'}



##### Create a new catalog with long display name

In [ ]:

import random
import string

ragHelper.log('Test - Create a new catalog with long display name')
longcatalogName = ''.join(random.choice(string.ascii_lowercase + string.digits) for _ in range(51)) #creating random string with 51 cahrs for long catalog name

ragHelper.log("Catalog Name: " + longcatalogName + '\n')

response = ragHelper.post("/catalog", {
    "displayName": longcatalogName
})

assert response.status_code == 400
assert (f"'{longcatalogName}' exceeds maximum length of 50") in response.json["details"][0]["Details"]

ragHelper.log('----------------------------------------------------------End of test----------------------------------------------------------')

Test - Create a new catalog with long display name
Test - Create a new catalog with long display name
Catalog Name: zhn31405qonvlgyweh9fnkv4n326tq66h9zjwunpq6wuwsha8ba

Request: POST https://api.qa.workbench.kpmg/rag/kpmg/catalog
Response: 400
{'error': 'Invalid JSON format', 'details': [{'Name': 'application/json', 'Type': 'RequestBody', 'ValidationRule': 'IncorrectMessage', 'Details': "Body of the request does not conform to the definition which is associated with the content type application/json. Path:displayName Message: String 'zhn31405qonvlgyweh9fnkv4n326tq66h9zjwunpq6wuwsha8ba' exceeds maximum length of 50. Line: 1, Position: 69 SchemaId: #/components/schemas/catalogRequest/properties/displayName", 'Action': 'Detected'}]}



##### Create a new catalog with empty display name

In [7]:
ragHelper.log('Test - Create a new catalog with empty display name')

emptycatalogName = f''

ragHelper.log("Catalog Name: " + emptycatalogName + '\n')

response = ragHelper.post("/catalog", {
    "displayName": emptycatalogName
})

assert response.status_code == 400
assert "The catalog name cannot be empty" in response.json["message"]

ragHelper.log('----------------------------------------------------------End of test----------------------------------------------------------')

Test - Create a new catalog with empty display name
Catalog Name: 

Request: POST https://api.qa.workbench.kpmg/rag/kpmg/catalog
Response: 400
{'message': 'The catalog name cannot be empty.'}

----------------------------------------------------------End of test----------------------------------------------------------


##### Update a catalog with non existing catalog

In [8]:
ragHelper.log('Test - Update a catalog with non existing catalog id')

nonExistCatalogId = 'ee3784f4-e570-46dc-bf1d-a64d8ed8bf55'
nonExistCatalogName = "Regression Catalog: non existing catalog"

ragHelper.log("Catalog Name: " + nonExistCatalogName + '\n')

response = ragHelper.put(f"/catalog/{nonExistCatalogId}", {
    "displayName": nonExistCatalogName
})

assert response.status_code == 404
assert 'The catalog was either not found or you are not the creator.' in response.json['message']

ragHelper.log('----------------------------------------------------------End of test----------------------------------------------------------')

Test - Update a catalog with non existing catalog id
Catalog Name: Regression Catalog: non existing catalog

Request: PUT https://api.qa.workbench.kpmg/rag/kpmg/catalog/ee3784f4-e570-46dc-bf1d-a64d8ed8bf55
Response: 404
{'message': 'The catalog was either not found or you are not the creator.'}

----------------------------------------------------------End of test----------------------------------------------------------


##### Get a catalog with non existing catalog

In [9]:
ragHelper.log('Test - Get a catalog with non existing catalog id')

nonExistCatalogId = 'ee3784f4-e570-46dc-bf1d-a64d8ed8bf55'
nonExistCatalogName = "Regression Catalog: non existing catalog"

ragHelper.log("Catalog Name: " + nonExistCatalogName + '\n')

response = ragHelper.get(f"/catalog/{nonExistCatalogId}", {
    "displayName": nonExistCatalogName
})

assert response.status_code == 404
assert 'The catalog was either not found or you are not the creator.' in response.json['message']

ragHelper.log('----------------------------------------------------------End of test----------------------------------------------------------')

Test - Get a catalog with non existing catalog id
Catalog Name: Regression Catalog: non existing catalog

Request: GET https://api.qa.workbench.kpmg/rag/kpmg/catalog/ee3784f4-e570-46dc-bf1d-a64d8ed8bf55
Response: 404
{'message': 'The catalog was either not found or you are not the creator.'}

----------------------------------------------------------End of test----------------------------------------------------------


##### Delete a catalog with non existing catalog

In [10]:
ragHelper.log('Test - Delete a catalog with non existing catalog id')

nonExistCatalogId = 'ee3784f4-e570-46dc-bf1d-a64d8ed8bf55'
nonExistCatalogName = "Regression Catalog: non existing catalog"

ragHelper.log("Catalog Name: " + nonExistCatalogName + '\n')

response = ragHelper.delete(f"/catalog/{nonExistCatalogId}", {
    })

assert response.status_code == 404
assert 'The catalogs are either not found or you are not the creator.' in response.json['message']

ragHelper.log('----------------------------------------------------------End of test----------------------------------------------------------')

Test - Delete a catalog with non existing catalog id
Catalog Name: Regression Catalog: non existing catalog

Request: DELETE https://api.qa.workbench.kpmg/rag/kpmg/catalog/ee3784f4-e570-46dc-bf1d-a64d8ed8bf55
Response: 404

----------------------------------------------------------End of test----------------------------------------------------------


#### Invalid subscription id - Catalog APIs

##### Create a new catalog with Invalid subscription key

In [11]:
ragHelper.log('Test - Create a new catalog with invalid subscription key')

ragHelper = Helper(local_env, "rag/kpmg", 'invalidkey', region_override)

response = ragHelper.post("/catalog", {
    "displayName": "Regression Catalog:Invalid subscription key"
})

assert response.status_code == 401
assert 'Access denied due to invalid subscription key. Make sure to provide a valid key for an active subscription.' in response.json['message']

ragHelper.log('----------------------------------------------------------End of test----------------------------------------------------------')

Test - Create a new catalog with invalid subscription key
Request: POST https://api.qa.workbench.kpmg/rag/kpmg/catalog
Response: 401
{'statusCode': 401, 'message': 'Access denied due to invalid subscription key. Make sure to provide a valid key for an active subscription.'}

----------------------------------------------------------End of test----------------------------------------------------------


##### Update catalog with Invalid subscription key

In [12]:
ragHelper.log('Test - Update a catalog with invalid subscription key')

response = ragHelper.put(f"/catalog/{catalogId}", {
    "displayName": "Regression Catalog:Invalid subscription key"
})

assert response.status_code == 401
assert 'Access denied due to invalid subscription key. Make sure to provide a valid key for an active subscription.' in response.json['message']

ragHelper.log('----------------------------------------------------------End of test----------------------------------------------------------')

Test - Update a catalog with invalid subscription key
Request: PUT https://api.qa.workbench.kpmg/rag/kpmg/catalog/d955e43b-920d-4d74-9c7b-13dc6b0c3498
Response: 401
{'statusCode': 401, 'message': 'Access denied due to invalid subscription key. Make sure to provide a valid key for an active subscription.'}

----------------------------------------------------------End of test----------------------------------------------------------


##### Get catalog with Invalid subscription key

In [13]:
ragHelper.log('Test - Get a catalog with invalid subscription key')

response = ragHelper.get(f"/catalog/{catalogId}", {
    "displayName": "Regression Catalog:Invalid subscription key"
})

assert response.status_code == 401
assert 'Access denied due to invalid subscription key. Make sure to provide a valid key for an active subscription.' in response.json['message']

ragHelper.log('----------------------------------------------------------End of test----------------------------------------------------------')

Test - Get a catalog with invalid subscription key
Request: GET https://api.qa.workbench.kpmg/rag/kpmg/catalog/d955e43b-920d-4d74-9c7b-13dc6b0c3498
Response: 401
{'statusCode': 401, 'message': 'Access denied due to invalid subscription key. Make sure to provide a valid key for an active subscription.'}

----------------------------------------------------------End of test----------------------------------------------------------


##### Delete catalog with Invalid subscription key

In [14]:
ragHelper.log('Test - Delete a catalog with invalid subscription key')

response = ragHelper.delete(f"/catalog/{catalogId}", {
    "displayName": "Regression Catalog:Invalid subscription key"
})

assert response.status_code == 401
assert 'Access denied due to invalid subscription key. Make sure to provide a valid key for an active subscription.' in response.json['message']

ragHelper.log('----------------------------------------------------------End of test----------------------------------------------------------')

Test - Delete a catalog with invalid subscription key
Request: DELETE https://api.qa.workbench.kpmg/rag/kpmg/catalog/d955e43b-920d-4d74-9c7b-13dc6b0c3498
Response: 401
{'statusCode': 401, 'message': 'Access denied due to invalid subscription key. Make sure to provide a valid key for an active subscription.'}

----------------------------------------------------------End of test----------------------------------------------------------


### Catalog Source Tests

##### Generate a SAS URI for the catalog source

In [15]:
ragHelper.log('Generating source SAS URI')

from azure.storage.blob import BlobServiceClient, generate_container_sas, ContainerSasPermissions
from datetime import datetime, timedelta, timezone

source_container_name = 'reg-source'
directory_name = 'sourceData'

blob_service_client = BlobServiceClient.from_connection_string(account_connection_string)
account_name = blob_service_client.account_name

source_sas_token = generate_container_sas(
    account_name=account_name,
    container_name=source_container_name,
    account_key=blob_service_client.credential.account_key,
    permission=ContainerSasPermissions(read=True, list=True),
    expiry=datetime.now(timezone.utc) + timedelta(hours=24)
)

source_sas_uri = f"https://{account_name}.blob.core.windows.net/{source_container_name}/{directory_name}?{source_sas_token}"

ragHelper.log("Source SAS URI: " + source_sas_uri)

Generating source SAS URI
Source SAS URI: https://dlsapimregwbameruseqa.blob.core.windows.net/reg-source/sourceData?se=2025-04-12T09%3A33%3A53Z&sp=rl&sv=2025-01-05&sr=c&sig=YEbakRmifXa7lnAFa%2B9%2Bj/RqJ5Lz8I%2BM6XtjN/QWfaU%3D


##### Create Catalog Source

In [17]:
ragHelper.log('Create a new catalog source for QA-Regression')

ragHelper = Helper(local_env, "rag/kpmg", api_key, region_override)
catalogSourceName = f'Regression Catalog Source'

response = ragHelper.post(f"/catalog/{catalogId}/source/blob", {
    "displayName": catalogSourceName,
    "uri": source_sas_uri
})

print(source_sas_uri)

assert response.status_code == 200

sourceId = response.json["sourceId"]

ragHelper.log("Created Catalog Source Id: " + sourceId)

Create a new catalog source for QA-Regression
Request: POST https://api.qa.workbench.kpmg/rag/kpmg/catalog/d955e43b-920d-4d74-9c7b-13dc6b0c3498/source/blob
Response: 200
{'sourceId': 'b93fde5c-0f80-4fa3-dc98-08dd78cac02e'}

https://dlsapimregwbameruseqa.blob.core.windows.net/reg-source/sourceData?se=2025-04-12T09%3A33%3A53Z&sp=rl&sv=2025-01-05&sr=c&sig=YEbakRmifXa7lnAFa%2B9%2Bj/RqJ5Lz8I%2BM6XtjN/QWfaU%3D
Created Catalog Source Id: b93fde5c-0f80-4fa3-dc98-08dd78cac02e


##### Create a catalog source with existing display name

In [18]:
ragHelper.log('Test - Create a new catalog source with existing display name')

catalogSourceName = f'Regression Catalog Source'   #this is existing source name

response = ragHelper.post(f"/catalog/{catalogId}/source/blob", {
    "displayName": catalogSourceName,
    "uri": source_sas_uri
})

print(source_sas_uri)

assert response.status_code == 409
assert (f"A catalog source with name '{catalogSourceName}' already exists") in response.json["message"]

ragHelper.log('----------------------------------------------------------End of test----------------------------------------------------------')

Test - Create a new catalog source with existing display name
Request: POST https://api.qa.workbench.kpmg/rag/kpmg/catalog/d955e43b-920d-4d74-9c7b-13dc6b0c3498/source/blob
Response: 409
{'message': "A catalog source with name 'Regression Catalog Source' already exists."}

https://dlsapimregwbameruseqa.blob.core.windows.net/reg-source/sourceData?se=2025-04-12T09%3A33%3A53Z&sp=rl&sv=2025-01-05&sr=c&sig=YEbakRmifXa7lnAFa%2B9%2Bj/RqJ5Lz8I%2BM6XtjN/QWfaU%3D
----------------------------------------------------------End of test----------------------------------------------------------


##### Create a catalog source with non existing catalog

In [19]:
ragHelper.log('Test - Create a new catalog source with non existing catalog id')

catalogSourceName = f'Regression Catalog Source'

response = ragHelper.post(f"/catalog/{nonExistCatalogId}/source/blob", {
    "displayName": catalogSourceName,
    "uri": source_sas_uri
})

print(source_sas_uri)

assert response.status_code == 404
assert "The catalog was either not found or you are not the creator" in response.json["message"]

ragHelper.log('----------------------------------------------------------End of test----------------------------------------------------------')

Test - Create a new catalog source with non existing catalog id
Request: POST https://api.qa.workbench.kpmg/rag/kpmg/catalog/ee3784f4-e570-46dc-bf1d-a64d8ed8bf55/source/blob
Response: 404
{'message': 'The catalog was either not found or you are not the creator.'}

https://dlsapimregwbameruseqa.blob.core.windows.net/reg-source/sourceData?se=2025-04-12T09%3A33%3A53Z&sp=rl&sv=2025-01-05&sr=c&sig=YEbakRmifXa7lnAFa%2B9%2Bj/RqJ5Lz8I%2BM6XtjN/QWfaU%3D
----------------------------------------------------------End of test----------------------------------------------------------


##### Create a catalog source with long name

In [20]:
ragHelper.log('Test - Create a new catalog source with long display name')

longCatalogSourceName = ''.join(random.choice(string.ascii_lowercase + string.digits) for _ in range(51)) #Catalog souce name with 51 chars
ragHelper.log("Catalog Source Name: " + longCatalogSourceName + '\n')

response = ragHelper.post(f"/catalog/{catalogId}/source/blob", {
    "displayName": longCatalogSourceName,
    "uri": source_sas_uri
})

assert response.status_code == 400
assert "A catalog name must not exceed 50 characters." in response.json["message"]

ragHelper.log('----------------------------------------------------------End of test----------------------------------------------------------')

Test - Create a new catalog source with long display name
Catalog Source Name: s1og460o89ymp731mmldfq8baurr0snfgdoz8xlu66rk4pijhej

Request: POST https://api.qa.workbench.kpmg/rag/kpmg/catalog/d955e43b-920d-4d74-9c7b-13dc6b0c3498/source/blob
Response: 400
{'message': 'A catalog name must not exceed 50 characters.'}

----------------------------------------------------------End of test----------------------------------------------------------


##### Create a catalog source with empty display name

In [21]:
ragHelper.log('Test - Create a new catalog source with empty display name')

emptyCatalogSourceName = f''
ragHelper.log("Catalog Source Name: " + emptyCatalogSourceName + '\n')

response = ragHelper.post(f"/catalog/{catalogId}/source/blob", {
    "displayName": emptyCatalogSourceName,
    "uri": source_sas_uri
})

assert response.status_code == 400
assert "A catalog name must be provided." in response.json["message"]

ragHelper.log('----------------------------------------------------------End of test----------------------------------------------------------')

Test - Create a new catalog source with empty display name
Catalog Source Name: 

Request: POST https://api.qa.workbench.kpmg/rag/kpmg/catalog/d955e43b-920d-4d74-9c7b-13dc6b0c3498/source/blob
Response: 400
{'message': 'A catalog name must be provided.'}

----------------------------------------------------------End of test----------------------------------------------------------


##### Create a catalog source with invalid sas uri

In [22]:
ragHelper.log('Test - Create a new catalog source with invalid sas uri')

catalogSourceName = f'Regression Catalog Source Invalid source sas'
invalid_source_sas_uri = 'https://dlsusewbqareg.blob.core.windows.net/reg-source/sourceData?se=2025-01-29T07%3A28%3A47Z&sp=rl&sv=2025-01-05&sr=c&sig=yOdf1qwbGJo9yUC7b8iragREG'

response = ragHelper.post(f"/catalog/{catalogId}/source/blob", {
    "displayName": catalogSourceName,
    "uri": invalid_source_sas_uri
})

assert response.status_code == 400
assert "SAS URI is expired. Please generate a new SAS token." in response.json["message"]

ragHelper.log('----------------------------------------------------------End of test----------------------------------------------------------')

Test - Create a new catalog source with invalid sas uri
Request: POST https://api.qa.workbench.kpmg/rag/kpmg/catalog/d955e43b-920d-4d74-9c7b-13dc6b0c3498/source/blob
Response: 400
{'message': 'SAS URI is expired. Please generate a new SAS token.'}

----------------------------------------------------------End of test----------------------------------------------------------


In [23]:
ragHelper.log('Test - Create a new catalog source with invalid sas uri (does not have http domain)')

invalid_source_sas_uritwo='//jhvjhvjhvjjhvj'

response = ragHelper.post(f"/catalog/{catalogId}/source/blob", {
    "displayName": catalogSourceName,
    "uri": invalid_source_sas_uritwo
})

assert response.status_code == 400
assert (f"'{invalid_source_sas_uritwo}' does not validate against format 'uri'") in response.json["details"][0]['Details']

ragHelper.log('----------------------------------------------------------End of test----------------------------------------------------------')

Test - Create a new catalog source with invalid sas uri (does not have http domain)
Request: POST https://api.qa.workbench.kpmg/rag/kpmg/catalog/d955e43b-920d-4d74-9c7b-13dc6b0c3498/source/blob
Response: 400
{'error': 'Invalid JSON format', 'details': [{'Name': 'application/json', 'Type': 'RequestBody', 'ValidationRule': 'IncorrectMessage', 'Details': "Body of the request does not conform to the definition which is associated with the content type application/json. Path:uri Message: String '//jhvjhvjhvjjhvj' does not validate against format 'uri'. Line: 1, Position: 89 SchemaId: #/components/schemas/blobCatalogSourceRequest/properties/uri", 'Action': 'Detected'}]}

----------------------------------------------------------End of test----------------------------------------------------------


##### Gets a catalog source with non existing catalog

In [24]:
ragHelper.log('Test - Get a catalog source with non existing catalog id')

CatalogSourceName = 'Regression Catalog Source'
ragHelper.log("Catalog Source Name: " + CatalogSourceName + '\n')

response = ragHelper.get(f"/catalog/{nonExistCatalogId}/source/{sourceId}", {
    "displayName": CatalogSourceName,
    "uri": source_sas_uri
})

assert response.status_code == 404
assert "The catalog source was either not found or you are not the creator." in response.json["message"]

ragHelper.log('----------------------------------------------------------End of test----------------------------------------------------------')

Test - Get a catalog source with non existing catalog id
Catalog Source Name: Regression Catalog Source

Request: GET https://api.qa.workbench.kpmg/rag/kpmg/catalog/ee3784f4-e570-46dc-bf1d-a64d8ed8bf55/source/b93fde5c-0f80-4fa3-dc98-08dd78cac02e
Response: 404
{'message': 'The catalog source was either not found or you are not the creator.'}

----------------------------------------------------------End of test----------------------------------------------------------


##### Gets a catalog source with non existing catalog source

In [25]:
ragHelper.log('Test - Get a catalog source with non existing source id')

CatalogSourceName = 'Regression Catalog Source'
nonExistCatalogSourceId='9a6c4f42-ef3d-4899-b5b4-08dd39d0bb6c'

ragHelper.log("Catalog Source Name: " + CatalogSourceName + '\n')

response = ragHelper.get(f"/catalog/{catalogId}/source/{nonExistCatalogSourceId}", {
    "displayName": CatalogSourceName,
    "uri": source_sas_uri
})

assert response.status_code == 404
assert "The catalog source was either not found or you are not the creator." in response.json["message"]

ragHelper.log('----------------------------------------------------------End of test----------------------------------------------------------')

Test - Get a catalog source with non existing source id
Catalog Source Name: Regression Catalog Source

Request: GET https://api.qa.workbench.kpmg/rag/kpmg/catalog/d955e43b-920d-4d74-9c7b-13dc6b0c3498/source/9a6c4f42-ef3d-4899-b5b4-08dd39d0bb6c
Response: 404
{'message': 'The catalog source was either not found or you are not the creator.'}

----------------------------------------------------------End of test----------------------------------------------------------


##### Updates a catalog source with non existing catalog

In [26]:
print('Test - Updates a catalog source with non existing catalog id')

CatalogSourceName = 'Regression Catalog Source'
ragHelper.log("Catalog Source Name: " + CatalogSourceName + '\n')

response = ragHelper.put(f"/catalog/{nonExistCatalogId}/source/{sourceId}", {
    "displayName": CatalogSourceName,
    "uri": source_sas_uri
})

assert response.status_code == 404
assert "The catalog source was either not found or you are not the creator." in response.json["message"]

ragHelper.log('----------------------------------------------------------End of test----------------------------------------------------------')

Test - Updates a catalog source with non existing catalog id
Catalog Source Name: Regression Catalog Source

Request: PUT https://api.qa.workbench.kpmg/rag/kpmg/catalog/ee3784f4-e570-46dc-bf1d-a64d8ed8bf55/source/b93fde5c-0f80-4fa3-dc98-08dd78cac02e
Response: 404
{'message': 'The catalog source was either not found or you are not the creator.'}

----------------------------------------------------------End of test----------------------------------------------------------


##### Updates a catalog source with non existing catalog source

In [27]:
ragHelper.log('Test - Create a catalog source with non existing source id')

CatalogSourceName = 'Regression Catalog Source'
nonExistCatalogSourceId='9a6c4f42-ef3d-4899-b5b4-08dd39d0bb6c'

ragHelper.log("Catalog Source Name: " + CatalogSourceName + '\n')

response = ragHelper.put(f"/catalog/{catalogId}/source/{nonExistCatalogSourceId}", {
    "displayName": CatalogSourceName,
    "uri": source_sas_uri
})

assert response.status_code == 404
assert "The catalog source was either not found or you are not the creator." in response.json["message"]

ragHelper.log('----------------------------------------------------------End of test----------------------------------------------------------')

Test - Create a catalog source with non existing source id
Catalog Source Name: Regression Catalog Source

Request: PUT https://api.qa.workbench.kpmg/rag/kpmg/catalog/d955e43b-920d-4d74-9c7b-13dc6b0c3498/source/9a6c4f42-ef3d-4899-b5b4-08dd39d0bb6c
Response: 404
{'message': 'The catalog source was either not found or you are not the creator.'}

----------------------------------------------------------End of test----------------------------------------------------------


##### Deletes a catalog source with non existing catalog

In [29]:
ragHelper.log('Test - Deletes a catalog source with non existing catalog id')

CatalogSourceName = 'Regression Catalog Source'
ragHelper.log("Catalog Source Name: " + CatalogSourceName + '\n')

response = ragHelper.delete(f"/catalog/{nonExistCatalogId}/source/{sourceId}", {
    #"displayName": CatalogSourceName,
    #"uri": source_sas_uri
})

assert response.status_code == 404
assert "The catalogs are either not found or you are not the creator." in response.json["message"]

ragHelper.log('----------------------------------------------------------End of test----------------------------------------------------------')

Test - Deletes a catalog source with non existing catalog id
Catalog Source Name: Regression Catalog Source

Request: DELETE https://api.qa.workbench.kpmg/rag/kpmg/catalog/ee3784f4-e570-46dc-bf1d-a64d8ed8bf55/source/b93fde5c-0f80-4fa3-dc98-08dd78cac02e
Response: 404

----------------------------------------------------------End of test----------------------------------------------------------


##### Deletes a catalog source with non existing catalog source

In [30]:
ragHelper.log('Test - Deletes a catalog source with non existing source id')

CatalogSourceName = 'Regression Catalog Source'
nonExistCatalogSourceId='9a6c4f42-ef3d-4899-b5b4-08dd39d0bb6c'

ragHelper.log("Catalog Source Name: " + CatalogSourceName + '\n')

response = ragHelper.delete(f"/catalog/{catalogId}/source/{nonExistCatalogSourceId}", {
    #"displayName": CatalogSourceName,
    #"uri": source_sas_uri
})

assert response.status_code == 404
assert "The catalogs are either not found or you are not the creator." in response.json["message"]

ragHelper.log('----------------------------------------------------------End of test----------------------------------------------------------')

Test - Deletes a catalog source with non existing source id
Catalog Source Name: Regression Catalog Source

Request: DELETE https://api.qa.workbench.kpmg/rag/kpmg/catalog/d955e43b-920d-4d74-9c7b-13dc6b0c3498/source/9a6c4f42-ef3d-4899-b5b4-08dd39d0bb6c
Response: 404

----------------------------------------------------------End of test----------------------------------------------------------


##### Gets all catalog sources with non existing catalog

In [31]:
ragHelper.log('Test - Get all catalog sources with non existing catalog id')

CatalogSourceName = 'Regression Catalog Source'
ragHelper.log("Catalog Source Name: " + CatalogSourceName + '\n')

response = ragHelper.get(f"/catalog/{nonExistCatalogId}/source", {
    "displayName": CatalogSourceName,
    "uri": source_sas_uri
})

assert response.status_code == 404
assert "The catalog sources are either not found or you are not the creator." in response.json["message"]

ragHelper.log('----------------------------------------------------------End of test----------------------------------------------------------')

Test - Get all catalog sources with non existing catalog id
Catalog Source Name: Regression Catalog Source

Request: GET https://api.qa.workbench.kpmg/rag/kpmg/catalog/ee3784f4-e570-46dc-bf1d-a64d8ed8bf55/source
Response: 404
{'message': 'The catalog sources are either not found or you are not the creator.'}

----------------------------------------------------------End of test----------------------------------------------------------


##### Gets all files in a catalog source with non existing catalog

In [32]:
ragHelper.log('Test - Get all files in a catalog source with non existing catalog id')

CatalogSourceName = 'Regression Catalog Source'
ragHelper.log("Catalog Source Name: " + CatalogSourceName + '\n')

response = ragHelper.get(f"/catalog/{nonExistCatalogId}/source/{sourceId}/files", {
    #"displayName": CatalogSourceName,
    #"uri": source_sas_uri
})

assert response.status_code == 404
assert "The catalog source was either not found or you are not the creator." in response.json["message"]

ragHelper.log('----------------------------------------------------------End of test----------------------------------------------------------')

Test - Get all files in a catalog source with non existing catalog id
Catalog Source Name: Regression Catalog Source

Request: GET https://api.qa.workbench.kpmg/rag/kpmg/catalog/ee3784f4-e570-46dc-bf1d-a64d8ed8bf55/source/b93fde5c-0f80-4fa3-dc98-08dd78cac02e/files
Response: 404

----------------------------------------------------------End of test----------------------------------------------------------


##### Gets all files in a catalog source with non existing source

In [33]:
ragHelper.log('Test - Get all files in  a catalog source with non existing source id')

CatalogSourceName = 'Regression Catalog Source'
nonExistCatalogSourceId='9a6c4f42-ef3d-4899-b5b4-08dd39d0bb6c'

ragHelper.log("Catalog Source Name: " + CatalogSourceName + '\n')

response = ragHelper.get(f"/catalog/{catalogId}/source/{nonExistCatalogSourceId}/files", {
    #"displayName": CatalogSourceName,
    #"uri": source_sas_uri
})

assert response.status_code == 404
assert "The catalog source was either not found or you are not the creator." in response.json["message"]

ragHelper.log('----------------------------------------------------------End of test----------------------------------------------------------')

Test - Get all files in  a catalog source with non existing source id
Catalog Source Name: Regression Catalog Source

Request: GET https://api.qa.workbench.kpmg/rag/kpmg/catalog/d955e43b-920d-4d74-9c7b-13dc6b0c3498/source/9a6c4f42-ef3d-4899-b5b4-08dd39d0bb6c/files
Response: 404

----------------------------------------------------------End of test----------------------------------------------------------


#### Invalid subscription id - Catalog Source APIs

##### Create a catalog source with invalid subscription id

In [34]:
ragHelper.log('Test - Create a new catalog source with invalid subscription id')

ragHelper = Helper(local_env, "rag/kpmg", 'invalidkey', region_override)
catalogSourceName = f'Regression Catalog Source'

response = ragHelper.post(f"/catalog/{catalogId}/source/blob", {
    "displayName": catalogSourceName,
    "uri": source_sas_uri
})


assert response.status_code == 401
assert 'Access denied due to invalid subscription key. Make sure to provide a valid key for an active subscription.' in response.json['message']

ragHelper.log('----------------------------------------------------------End of test----------------------------------------------------------')

Test - Create a new catalog source with invalid subscription id
Request: POST https://api.qa.workbench.kpmg/rag/kpmg/catalog/d955e43b-920d-4d74-9c7b-13dc6b0c3498/source/blob
Response: 401
{'statusCode': 401, 'message': 'Access denied due to invalid subscription key. Make sure to provide a valid key for an active subscription.'}

----------------------------------------------------------End of test----------------------------------------------------------


##### Gets a catalog source with invalid subscription id

In [35]:
ragHelper.log('Test - Get a catalog source with invalid subscription id')

catalogSourceName = f'Regression Catalog Source'

response = ragHelper.get(f"/catalog/{catalogId}/source/{sourceId}", {
    "displayName": CatalogSourceName,
    "uri": source_sas_uri
})

assert response.status_code == 401
assert 'Access denied due to invalid subscription key. Make sure to provide a valid key for an active subscription.' in response.json['message']

ragHelper.log('----------------------------------------------------------End of test----------------------------------------------------------')

Test - Get a catalog source with invalid subscription id
Request: GET https://api.qa.workbench.kpmg/rag/kpmg/catalog/d955e43b-920d-4d74-9c7b-13dc6b0c3498/source/b93fde5c-0f80-4fa3-dc98-08dd78cac02e
Response: 401
{'statusCode': 401, 'message': 'Access denied due to invalid subscription key. Make sure to provide a valid key for an active subscription.'}

----------------------------------------------------------End of test----------------------------------------------------------


##### Updates a catalog source with invalid subscription id

In [36]:
ragHelper.log('Test - Update a catalog source with invalid subscription id')

catalogSourceName = f'Regression Catalog Source'

response = ragHelper.put(f"/catalog/{catalogId}/source/{sourceId}", {
    "displayName": CatalogSourceName,
    "uri": source_sas_uri
})

assert response.status_code == 401
assert 'Access denied due to invalid subscription key. Make sure to provide a valid key for an active subscription.' in response.json['message']

ragHelper.log('----------------------------------------------------------End of test----------------------------------------------------------')

Test - Update a catalog source with invalid subscription id
Request: PUT https://api.qa.workbench.kpmg/rag/kpmg/catalog/d955e43b-920d-4d74-9c7b-13dc6b0c3498/source/b93fde5c-0f80-4fa3-dc98-08dd78cac02e
Response: 401
{'statusCode': 401, 'message': 'Access denied due to invalid subscription key. Make sure to provide a valid key for an active subscription.'}

----------------------------------------------------------End of test----------------------------------------------------------


##### Deletes a catalog source with invalid subscription id

In [37]:
ragHelper.log('Test - Delete a catalog source with invalid subscription id')

catalogSourceName = f'Regression Catalog Source'

response = ragHelper.delete(f"/catalog/{catalogId}/source/{sourceId}", {
    #"displayName": CatalogSourceName,
    #"uri": source_sas_uri
})

assert response.status_code == 401
assert 'Access denied due to invalid subscription key. Make sure to provide a valid key for an active subscription.' in response.json['message']

ragHelper.log('----------------------------------------------------------End of test----------------------------------------------------------')

Test - Delete a catalog source with invalid subscription id
Request: DELETE https://api.qa.workbench.kpmg/rag/kpmg/catalog/d955e43b-920d-4d74-9c7b-13dc6b0c3498/source/b93fde5c-0f80-4fa3-dc98-08dd78cac02e
Response: 401

----------------------------------------------------------End of test----------------------------------------------------------


##### Gets all catalog source with invalid subscription id

In [38]:
ragHelper.log('Test - Get  all catalog sources with invalid subscription id')

catalogSourceName = f'Regression Catalog Source'

response = ragHelper.get(f"/catalog/{catalogId}/source", {
    "displayName": CatalogSourceName,
    "uri": source_sas_uri
})

assert response.status_code == 401
assert 'Access denied due to invalid subscription key. Make sure to provide a valid key for an active subscription.' in response.json['message']

ragHelper.log('----------------------------------------------------------End of test----------------------------------------------------------')

Test - Get  all catalog sources with invalid subscription id
Request: GET https://api.qa.workbench.kpmg/rag/kpmg/catalog/d955e43b-920d-4d74-9c7b-13dc6b0c3498/source
Response: 401
{'statusCode': 401, 'message': 'Access denied due to invalid subscription key. Make sure to provide a valid key for an active subscription.'}

----------------------------------------------------------End of test----------------------------------------------------------


##### Gets all files in a catalog source with invalid subscription id

In [39]:
ragHelper.log('Test - Get all files in a catalog source with invalid subscription id')

catalogSourceName = f'Regression Catalog Source'

response = ragHelper.get(f"/catalog/{catalogId}/source/{sourceId}/files", {
    #"displayName": CatalogSourceName,
    #"uri": source_sas_uri
})

assert response.status_code == 401
assert 'Access denied due to invalid subscription key. Make sure to provide a valid key for an active subscription.' in response.json['message']

ragHelper.log('----------------------------------------------------------End of test----------------------------------------------------------')

Test - Get all files in a catalog source with invalid subscription id
Request: GET https://api.qa.workbench.kpmg/rag/kpmg/catalog/d955e43b-920d-4d74-9c7b-13dc6b0c3498/source/b93fde5c-0f80-4fa3-dc98-08dd78cac02e/files
Response: 401

----------------------------------------------------------End of test----------------------------------------------------------


### Ingestion Tests

##### Start an Ingestion Job

In [40]:
ragHelper.log('Start an ingestion job for QA-Regression')

ragHelper = Helper(local_env, "rag/kpmg", api_key, region_override)

response = ragHelper.post(f"/catalog/{catalogId}/ingest", None)

assert response.status_code == 202
assert 'ingestionJobId' in response.json

ingestionJobId = response.json["ingestionJobId"]

ragHelper.log("Created Ingestion Job Id: " + ingestionJobId)

Start an ingestion job for QA-Regression
Request: POST https://api.qa.workbench.kpmg/rag/kpmg/catalog/d955e43b-920d-4d74-9c7b-13dc6b0c3498/ingest
Response: 202
{'ingestionJobId': 'b7539618-214e-4a06-aae1-4b0eb2b6671c', 'createDate': '2025-04-11T10:13:05.4788075Z', 'ingestionStatus': 'Ingesting'}

Created Ingestion Job Id: b7539618-214e-4a06-aae1-4b0eb2b6671c


##### Starts an Ingestion job with non existing catalog 

In [42]:
ragHelper.log('Test - Start an ingestion with non existing catalog id')

response = ragHelper.post(f"/catalog/{nonExistCatalogId}/ingest", None)

assert response.status_code == 404
assert "The catalog was either not found or you are not the creator." in response.json["message"]

ragHelper.log('----------------------------------------------------------End of test----------------------------------------------------------')

Test - Start an ingestion with non existing catalog id
Request: POST https://api.qa.workbench.kpmg/rag/kpmg/catalog/ee3784f4-e570-46dc-bf1d-a64d8ed8bf55/ingest
Response: 404
{'message': 'The catalog was either not found or you are not the creator.'}

----------------------------------------------------------End of test----------------------------------------------------------


##### Get the Status of the Job

In [43]:
ragHelper.log('Positive test to get status of ingestion job')

import time

ingestionStatus = 'Ingesting'
waitTime = 5

while (ingestionStatus != 'Ingested' and ingestionStatus != 'IngestionFailed'):
    time.sleep(waitTime)
    response = ragHelper.get(f"/catalog/{catalogId}/ingest/{ingestionJobId}")

    ingestionStatus = response.json["ingestionStatus"]

    if (waitTime < 30):
        # backoff polling wait time
        # the longer we wait, the less frequently we poll
        waitTime = waitTime + 5

ragHelper.log(f"Ingestion Job Status: {ingestionStatus}")

#assert response.status_code == 200
#assert ingestionStatus == 'Ingested'

Positive test to get status of ingestion job
Request: GET https://api.qa.workbench.kpmg/rag/kpmg/catalog/d955e43b-920d-4d74-9c7b-13dc6b0c3498/ingest/b7539618-214e-4a06-aae1-4b0eb2b6671c
Response: 200
{'ingestionJobId': 'b7539618-214e-4a06-aae1-4b0eb2b6671c', 'ingestionStatus': 'Ingesting', 'lastModifiedDate': '2025-04-11T10:13:29.6146983Z', 'createDate': '2025-04-11T10:13:05.6109532', 'duration': '00:00:24.0037451'}

Request: GET https://api.qa.workbench.kpmg/rag/kpmg/catalog/d955e43b-920d-4d74-9c7b-13dc6b0c3498/ingest/b7539618-214e-4a06-aae1-4b0eb2b6671c
Response: 200
{'ingestionJobId': 'b7539618-214e-4a06-aae1-4b0eb2b6671c', 'ingestionStatus': 'Ingested', 'lastModifiedDate': '2025-04-11T10:13:43.0467719Z', 'createDate': '2025-04-11T10:13:05.6109532', 'duration': '00:00:37.4358187'}

Ingestion Job Status: Ingested


##### Gets the status of Ingestion job with non existing catalog id

In [44]:
ragHelper.log('Test - Get the status of ingestion job with non existing catalog id')

response = ragHelper.get(f"/catalog/{nonExistCatalogId}/ingest/{ingestionJobId}")

assert response.status_code == 404
assert "The catalog was either not found or you are not the creator." in response.json["message"]

ragHelper.log('----------------------------------------------------------End of test----------------------------------------------------------')

Test - Get the status of ingestion job with non existing catalog id
Request: GET https://api.qa.workbench.kpmg/rag/kpmg/catalog/ee3784f4-e570-46dc-bf1d-a64d8ed8bf55/ingest/b7539618-214e-4a06-aae1-4b0eb2b6671c
Response: 404
{'message': 'The catalog was either not found or you are not the creator.'}

----------------------------------------------------------End of test----------------------------------------------------------


##### Gets the status of Ingestion job with non existing ingestion job id

In [45]:
ragHelper.log('Test - Get the status of ingestion job with non existing ingestion job id')

nonExistingIngestionId='697749d1-1008-49aa-a58c-f0e11c2bbe78'

response = ragHelper.get(f"/catalog/{catalogId}/ingest/{nonExistingIngestionId}")

assert response.status_code == 404
assert "Ingestion job was not found." in response.json["message"]

ragHelper.log('----------------------------------------------------------End of test----------------------------------------------------------')


Test - Get the status of ingestion job with non existing ingestion job id
Request: GET https://api.qa.workbench.kpmg/rag/kpmg/catalog/d955e43b-920d-4d74-9c7b-13dc6b0c3498/ingest/697749d1-1008-49aa-a58c-f0e11c2bbe78
Response: 404
{'message': 'Ingestion job was not found.'}

----------------------------------------------------------End of test----------------------------------------------------------


##### Get Ingestion Files

In [46]:
ragHelper.log('Get ingestion files')

from tabulate import tabulate

response = ragHelper.get(f"/catalog/{catalogId}/ingest/{ingestionJobId}/files")

file_list = ragHelper.get_property_values(response.json, 'fileName')

print(tabulate(response.json['value'], headers="keys", tablefmt="grid"))

ragHelper.log('----------------------------------------------------------End of test----------------------------------------------------------')

Get ingestion files
Request: GET https://api.qa.workbench.kpmg/rag/kpmg/catalog/d955e43b-920d-4d74-9c7b-13dc6b0c3498/ingest/b7539618-214e-4a06-aae1-4b0eb2b6671c/files
Response: 200
{'value': [{'ingestionJobId': 'b7539618-214e-4a06-aae1-4b0eb2b6671c', 'fileId': 'e7d2cdd9-50ba-4f01-2a6f-08dd78cac1c6', 'fileName': 'Best Practices for Application Logging.pdf', 'fileUri': 'https://dlsapimregwbameruseqa.blob.core.windows.net/reg-source/sourceData/Best Practices for Application Logging.pdf', 'fileMD5Hash': 'CB3D3190ACBCA1E97F9C21616D800014', 'fileStatus': 'Indexed', 'statusLastModifiedDate': '2025-04-11T10:13:42.730887'}, {'ingestionJobId': 'b7539618-214e-4a06-aae1-4b0eb2b6671c', 'fileId': 'c6240aa0-ceee-470c-2a70-08dd78cac1c6', 'fileName': 'Best Practices for Architect Engagement.pdf', 'fileUri': 'https://dlsapimregwbameruseqa.blob.core.windows.net/reg-source/sourceData/Best Practices for Architect Engagement.pdf', 'fileMD5Hash': '6D27BA108C543D7648DDD3DC261A2BB3', 'fileStatus': 'Indexed', '

##### Get Ingestion Files with non existing catalog

In [47]:
ragHelper.log('Test - Get ingestion files with non existing catalog id')

from tabulate import tabulate

response = ragHelper.get(f"/catalog/{nonExistCatalogId}/ingest/{ingestionJobId}/files")

#file_list = ragHelper.get_property_values(response.json, 'fileName')
#print(tabulate(response.json['value'], headers="keys", tablefmt="grid"))

assert response.status_code == 404
assert "The catalog was either not found or you are not the creator." in response.json["message"]

ragHelper.log('----------------------------------------------------------End of test----------------------------------------------------------')

Test - Get ingestion files with non existing catalog id
Request: GET https://api.qa.workbench.kpmg/rag/kpmg/catalog/ee3784f4-e570-46dc-bf1d-a64d8ed8bf55/ingest/b7539618-214e-4a06-aae1-4b0eb2b6671c/files
Response: 404
{'message': 'The catalog was either not found or you are not the creator.'}

----------------------------------------------------------End of test----------------------------------------------------------


##### Get Ingestion Files with non existing ingestion

In [48]:
ragHelper.log('Test - Get ingestion files with non existing ingestion job id')

from tabulate import tabulate

response = ragHelper.get(f"/catalog/{catalogId}/ingest/{nonExistingIngestionId}/files")

#file_list = ragHelper.get_property_values(response.json, 'fileName')
#print(tabulate(response.json['value'], headers="keys", tablefmt="grid"))

assert response.status_code == 404
assert "The specified job does not exist." in response.json["message"]

ragHelper.log('----------------------------------------------------------End of test----------------------------------------------------------')

Test - Get ingestion files with non existing ingestion job id
Request: GET https://api.qa.workbench.kpmg/rag/kpmg/catalog/d955e43b-920d-4d74-9c7b-13dc6b0c3498/ingest/697749d1-1008-49aa-a58c-f0e11c2bbe78/files
Response: 404
{'message': 'The specified job does not exist.'}

----------------------------------------------------------End of test----------------------------------------------------------


##### Get ingestion jobs for a catalog

In [49]:
ragHelper.log('Test - Get ingestion jobs for catalog')

response = ragHelper.get(f"/catalog/{catalogId}/ingest")

assert response.status_code == 200

Test - Get ingestion jobs for catalog
Request: GET https://api.qa.workbench.kpmg/rag/kpmg/catalog/d955e43b-920d-4d74-9c7b-13dc6b0c3498/ingest
Response: 200
[{'ingestionJobId': 'b7539618-214e-4a06-aae1-4b0eb2b6671c', 'createDate': '2025-04-11T10:13:05.6109532', 'ingestionStatus': 'Ingested'}]



##### Get ingestion jobs for a non existing catalog

In [50]:
ragHelper.log('Test - Get ingestion jobss with non existing catalog id')

response = ragHelper.get(f"/catalog/{nonExistCatalogId}/ingest")

assert response.status_code == 404
assert "The catalog was either not found or you are not the creator." in response.json["message"]

ragHelper.log('----------------------------------------------------------End of test----------------------------------------------------------')


Test - Get ingestion jobss with non existing catalog id
Request: GET https://api.qa.workbench.kpmg/rag/kpmg/catalog/ee3784f4-e570-46dc-bf1d-a64d8ed8bf55/ingest
Response: 404
{'message': 'The catalog was either not found or you are not the creator.'}

----------------------------------------------------------End of test----------------------------------------------------------


#### Invalid subscription id - Ingestion APIs

##### Start a new ingestion job ith invalid key

In [51]:
ragHelper.log('Test - Start a new ingestion with invalid subscription key')

ragHelper = Helper(local_env, "rag/kpmg", 'invalidkey', region_override)

response = ragHelper.post(f"/catalog/{catalogId}/ingest", None)


assert response.status_code == 401
assert 'Access denied due to invalid subscription key. Make sure to provide a valid key for an active subscription.' in response.json['message']

ragHelper.log('----------------------------------------------------------End of test----------------------------------------------------------')

Test - Start a new ingestion with invalid subscription key
Request: POST https://api.qa.workbench.kpmg/rag/kpmg/catalog/d955e43b-920d-4d74-9c7b-13dc6b0c3498/ingest
Response: 401
{'statusCode': 401, 'message': 'Access denied due to invalid subscription key. Make sure to provide a valid key for an active subscription.'}

----------------------------------------------------------End of test----------------------------------------------------------


##### Get the status of ingestion job ith invalid key

In [52]:
ragHelper.log('Test - Get the status ingestion job with invalid subscription key')

response = ragHelper.get(f"/catalog/{catalogId}/ingest/{ingestionJobId}")

assert response.status_code == 401
assert 'Access denied due to invalid subscription key. Make sure to provide a valid key for an active subscription.' in response.json['message']

ragHelper.log('----------------------------------------------------------End of test----------------------------------------------------------')

Test - Get the status ingestion job with invalid subscription key
Request: GET https://api.qa.workbench.kpmg/rag/kpmg/catalog/d955e43b-920d-4d74-9c7b-13dc6b0c3498/ingest/b7539618-214e-4a06-aae1-4b0eb2b6671c
Response: 401
{'statusCode': 401, 'message': 'Access denied due to invalid subscription key. Make sure to provide a valid key for an active subscription.'}

----------------------------------------------------------End of test----------------------------------------------------------


##### Gets files in ingestion job ith invalid key

In [53]:
ragHelper.log('Test - Get files in ingestion with invalid subscription key')

response = ragHelper.get(f"/catalog/{catalogId}/ingest/{ingestionJobId}/files")

assert response.status_code == 401
assert 'Access denied due to invalid subscription key. Make sure to provide a valid key for an active subscription.' in response.json['message']

ragHelper.log('----------------------------------------------------------End of test----------------------------------------------------------')

Test - Get files in ingestion with invalid subscription key
Request: GET https://api.qa.workbench.kpmg/rag/kpmg/catalog/d955e43b-920d-4d74-9c7b-13dc6b0c3498/ingest/b7539618-214e-4a06-aae1-4b0eb2b6671c/files
Response: 401
{'statusCode': 401, 'message': 'Access denied due to invalid subscription key. Make sure to provide a valid key for an active subscription.'}

----------------------------------------------------------End of test----------------------------------------------------------


##### Gets ingestion jobs for a catalog with invalid key

In [54]:
ragHelper.log('Test - Get ingestion jobs for a catalog with invalid subscription key')

response = ragHelper.get(f"/catalog/{catalogId}/ingest")

assert response.status_code == 401
assert 'Access denied due to invalid subscription key. Make sure to provide a valid key for an active subscription.' in response.json['message']

ragHelper.log('----------------------------------------------------------End of test----------------------------------------------------------')

Test - Get ingestion jobs for a catalog with invalid subscription key
Request: GET https://api.qa.workbench.kpmg/rag/kpmg/catalog/d955e43b-920d-4d74-9c7b-13dc6b0c3498/ingest
Response: 401
{'statusCode': 401, 'message': 'Access denied due to invalid subscription key. Make sure to provide a valid key for an active subscription.'}

----------------------------------------------------------End of test----------------------------------------------------------
